# Component aggregation

In [ ]:
from pyapi_rts.generated.lfrtdssharcsldSHUNTCAP import (
    lfrtdssharcsldSHUNTCAP,
)
from pyapi_rts.generated.rtdsudcDYLOAD import (
    rtdsudcDYLOAD,
)

from pyapi_rts.api import Draft, Component


def sum_by_key(cs: list[Component], key: str) -> float:
    # Sums up the values of a component's attribute
    return sum(map((lambda c: c.get_by_key(key)), cs))

draft = Draft()
draft.read_file("kit_aggregation_grouped.dfx")

B123_4_400V = next(
    filter(
        (lambda c: c.name == "B123_4_400V"),
        draft.subsystems[0].get_components(),
    ),
    None,
)
# Get relevant components
dyloads = list(
    filter(
        (lambda c: isinstance(c, rtdsudcDYLOAD)),
        draft.subsystems[0].get_connected_to(B123_4_400V),
    )
)
print(f"Found {len(dyloads)} Dyload components")
shunts = list(
    filter(
        (lambda c: isinstance(c, lfrtdssharcsldSHUNTCAP)),
        draft.subsystems[0].get_connected_to(B123_4_400V),
    )
)
print(f"Found {len(shunts)} Shunt components")

graph = draft.subsystems[0].get_connection_graph()

# Sum up the values of the dyload, write to first dyload
dyl: rtdsudcDYLOAD = dyloads[0].duplicate()
dyl.set_by_key("Qinit", sum_by_key(dyloads, "Qinit"))
dyl.set_by_key("Pinit", sum_by_key(dyloads, "Pinit"))
draft.subsystems[0].modify_component(dyl)
for dyload in dyloads[1:]:
    for neighbor in graph.neighbors(dyload.uuid):
        # Remove buses and wires connected to the dyload
        draft.subsystems[0].remove_component(neighbor, False)
    draft.subsystems[0].remove_component(dyload.uuid, False)
    print(f"Removed Dyload with UUID {dyload.uuid}")

# Sum up the values of the shunts, write to first shunt
shunt: lfrtdssharcsldSHUNTCAP = shunts[0].duplicate()
shunt.set_by_key("CuF", sum_by_key(shunts, "CuF"))
draft.subsystems[0].modify_component(shunt)

for shunt in shunts[1:]:
    draft.subsystems[0].remove_component(shunt.uuid, False)
    for neighbor in graph.neighbors(shunt.uuid):
        # Remove buses connected to the shunt
        draft.subsystems[0].remove_component(neighbor, False)
    print(f"Removed Shunt with UUID {neighbor}")

# Remove wirelabels not connected to anything
for wirelabel in draft.get_components_by_type("wirelabel", False):
    if len(draft.subsystems[0].get_connected_to(wirelabel)) == 1:
        # Only connected to itself, remove
        draft.subsystems[0].remove_component(wirelabel.uuid, False)

draft.write_file("aggregation_out.dfx")

print(f"Components: {len(draft.get_components())}")
print(f"Qinit sum: {dyl.PANDQSETTINGS.Qinit.get_value()}")
print(f"Pinit sum: {dyl.PANDQSETTINGS.Pinit.get_value()}")
print(f"CuF sum: {shunt.CONFIGURATION.CuF.get_value()}")

## Visualization

Look at the connection graph before and after the script

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

draft_old = Draft()
draft_old.read_file("kit_aggregation_grouped.dfx")

G_old = draft_old.subsystems[0].get_connection_graph()

color_map_old = []
for node in G_old.nodes():
    if isinstance(draft_old.get_by_id(node), rtdsudcDYLOAD):
        color_map_old.append("red")
    elif isinstance(draft_old.get_by_id(node), lfrtdssharcsldSHUNTCAP):
        color_map_old.append("blue")
    else:
        color_map_old.append("green")

nx.draw(G_old, with_labels=False, node_color=color_map_old)
plt.show()

G_new = draft.subsystems[0].get_connection_graph()

color_map_new = []
for node in G_new.nodes():
    if isinstance(draft.get_by_id(node), rtdsudcDYLOAD):
        color_map_new.append("red")
    elif isinstance(draft.get_by_id(node), lfrtdssharcsldSHUNTCAP):
        color_map_new.append("blue")
    else:
        color_map_new.append("green")
nx.draw(G_new, with_labels=False, node_color=color_map_new)
plt.show()